### Install packages

In [ ]:
!pip install gunpowder
!pip install zarr

### import packages

In [2]:
import gunpowder as gp
import os
import glob
import numpy as np
import zarr
import tifffile

2022-09-02 13:39:13.030885: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
### Convert data into zarr format
def tif2zarr(baseDir,outDir='',chunkSize=48,owrite=False):
    if outDir=='':
        outDir = baseDir+"/zarr/"
    
    if not os.path.isdir(outDir) or owrite:
        # load all tiff files 
        img = np.stack([tifffile.imread(xi) for xi in sorted(glob.glob(baseDir+"/"+"*.tif"))])  # images

        # format: [timepoints,zslices,y,z]
        imDims = img.shape
    
        # define chunk size
        if isinstance(chunkSize, int):
            chunks = np.repeat(chunkSize,len(imDims))
        else:
            chunks = chunkSize
                  
        # write img into zarr file       
        zFile = zarr.open(outDir, shape=imDims, chunks=chunks, mode="w", dtype='int32')
        zFile[:] = img
        
        return outDir
    else:
        return 'zarr file already present'

In [6]:
dataDir = "/mnt/shared/celltracking/data/cho/01"
test = tif2zarr(dataDir,owrite=True)
print(f'zarr data directory:{test}')

zarr data directory:/mnt/shared/celltracking/data/cho/01/zarr/


### Import data with gunpowder

In [7]:
# read data from zarr file
#imgs = zarr.open('data/example.zarr', mode='r')
#raw = gp.ArrayKey('RAW')
raw = gp.ZarrSource('/mnt/shared/celltracking/data/cho/01/zarr', datasets, array_specs=None, channels_first=True)

NameError: name 'datasets' is not defined

### Create list of cell pairs

In [ ]:
## Function to extract trajectories from data

# Input: image stack 

# options
#	paired=True
#	

# Function to generate random pairs of cells from t and t+1

# 

### Load volumes with gunpowder